In [2]:
# 📦 1. IMPORTS AND ENVIRONMENT SETUP
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from datetime import datetime
from zumi.util.vision import Vision
from zumi.util.camera import Camera 
from zumi.personality import Personality
import time
import pandas as pd
from collections import defaultdict
import cv2

In [ ]:
# ➡️ 2. EVENT LOGGER CLASS

class EventLogger:
    """
    Records events (action names) with timestamps in memory.
    On request, exports data to a CSV file.

    Attributes:
        log (defaultdict[list]): maps action (str) → list of datetime timestamps.
    """

    def __init__(self):
        self.log = defaultdict(list)

    def log_event(self, action: str):
        """
        Add the event 'action' with the current timestamp to the internal log.
        """
        self.log[action].append(datetime.now())

    def save_to_csv(self, output_folder: str = "."):
        """
        Save all logged events into a CSV file under `output_folder`.
        Filename format: “zumi_events_<YYYYMMDD_HHMMSS>.csv”.
        """
        rows = []
        for action, timestamps in self.log.items():
            for ts in timestamps:
                rows.append({"timestamp": ts, "action": action})

        df = pd.DataFrame(rows)
        if not df.empty:
            df = df.sort_values(by="timestamp")
            current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = os.path.join(output_folder, f"zumi_events_{current_time}.csv")
            df.to_csv(file_name, index=False)
            print(f"Events saved to {file_name}")
        else:
            print("No events to save.")

In [ ]:
# 🤖 3. ZUMI DRIVER CLASS

class ZumiDriver:
    """
    Controls the Zumi robot’s movement, line-following, and turning operations.

    Dependencies:
        - zumi: object providing low-level Zumi API (movement, sensors, gyro).
        - camera: object for camera control (capture, show_image, close).
        - vision: object for computer vision tasks (QR, face detection).
        - screen: object for drawing text/images on Zumi’s screen.
        - personality: object for setting Zumi’s mood (happy, angry, celebrate).
        - logger: instance of EventLogger to record key events.
    """

    def __init__(self, zumi, camera, vision, screen, personality, logger):
        self.zumi = zumi
        self.camera = camera
        self.vision = vision
        self.screen = screen
        self.personality = personality
        self.logger = logger

    def read_ir(self):
        """
        Read IR sensor values from Zumi.
        Returns:
            Tuple[int, int, int, int, int, int]:
            (front_right, bottom_right, back_right, bottom_left, back_left, front_left)
        """
        return self.zumi.get_all_IR_data()

    def line_correction(self, bottom_left, bottom_right, desired_angle, threshold):
        """
        Adjust desired_angle based on bottom IR sensors:
        - If bottom_left > threshold and bottom_right < threshold: desired_angle += 5
        - If bottom_left < threshold and bottom_right > threshold: desired_angle -= 5
        In both cases, stop for a brief moment for stability.
        """
        if bottom_left > threshold and bottom_right < threshold:
            desired_angle += 5
            self.zumi.stop()
            time.sleep(0.01)
        elif bottom_left < threshold and bottom_right > threshold:
            desired_angle -= 5
            self.zumi.stop()
            time.sleep(0.01)
        return desired_angle

    def turning_correction(self, desired_angle, turn_angle):
        """
        Simplify correction: compute delta = |turn_angle - |desired_angle||,
        then restore sign: return -delta if desired_angle < 0 else +delta.
        """
        delta = abs(turn_angle - abs(desired_angle))
        return -delta if desired_angle < 0 else delta

    def turn_90(self, direction):
        """
        Turn Zumi exactly 90 degrees in `direction` ('left' or 'right'):
        - Reset gyro, signal turn, execute 90°, stop signal, log event.
        """
        self.zumi.reset_gyro()
        if direction == "left":
            self.zumi.signal_left_on()
            self.zumi.turn_left(90)
            self.zumi.signal_left_off()
            self.logger.log_event("move_left")
        elif direction == "right":
            self.zumi.signal_right_on()
            self.zumi.turn_right(90)
            self.zumi.signal_right_off()
            self.logger.log_event("move_right")

    def turn_to_check(self, direction, angle=90):
        """
        Partially turn to check for a line:
        - If direction == 'left', turn left by `angle`.
        - If direction == 'right', turn right by `2 * angle`.
        Then return the gyro reading.
        """
        self.zumi.reset_gyro()
        if direction == "left":
            self.zumi.signal_left_on()
            self.zumi.turn_left(angle)
            self.zumi.signal_left_off()
        else:
            self.zumi.signal_right_on()
            self.zumi.turn_right(2 * angle)
            self.zumi.signal_right_off()
        time.sleep(0.01)
        return self.zumi.read_z_angle()

    def move_after_turn(self, speed, desired_angle, times=3):
        """
        After realigning heading, move straight for `times` iterations,
        then call stop().
        """
        self.zumi.reset_gyro()
        for _ in range(times):
            self.zumi.go_straight(speed, desired_angle)
        self.zumi.stop()

    def approach_line_front(self, speed, threshold):
        """
        Move forward until both bottom sensors detect the line (values < threshold).
        """
        while True:
            _, bottom_right, _, bottom_left, _, _ = self.read_ir()
            self.zumi.go_straight(speed, 0)
            if bottom_right < threshold and bottom_left < threshold:
                self.zumi.stop()
                break

    def follow_line_until_loss(self, speed, initial_angle, threshold):
        """
        Follow the line while either bottom sensor is on-line (value < threshold).
        When both bottom sensors > threshold → exit and stop().
        """
        desired_angle = initial_angle
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            if br > threshold and bl > threshold:
                self.zumi.stop()
                break
            desired_angle = self.turning_correction(desired_angle, 90)
            self.zumi.go_straight(speed, desired_angle)

    def circle(self, direction, number_of_objects, threshold, speed=4):
        """
        Drive a square 'circle' around each detected line segment:
        1. Approach the line from front.
        2. For each object, do 4 разів: turn_90 + follow_line_until_loss.
        """
        self.logger.log_event(f"start_{direction}_circle")
        self.zumi.reset_gyro()

        # Step 1: get on the line frontally
        self.approach_line_front(speed, threshold)

        # Step 2: for each object, perform 4 сегменти квадрата
        for _ in range(number_of_objects):
            for _ in range(4):
                self.turn_90(direction)
                desired_angle = self.zumi.read_z_angle()
                desired_angle = self.turning_correction(desired_angle, 90)
                self.follow_line_until_loss(speed, desired_angle, threshold)

        self.logger.log_event(f"done_{direction}_circle")

    def find_finish_line(self):
        """
        Detect final finish line and realign:
        - Try turn left (80°). If no line, try turn right (80°).
        - Once on finish line, drive forward until both bottom sensors > threshold.
        - Then move_after_turn and final 25° turn.
        """
        speed = 3
        threshold = 100
        self.zumi.reset_gyro()

        # Try turn left side
        turned_left_angle = self.turn_to_check("left", 80)
        _, br, _, bl, _, _ = self.read_ir()

        if bl > threshold and br > threshold:
            # Found line on left
            print("line left")
            while True:
                self.zumi.turn_right(5)
                _, br, _, bl, _, _ = self.read_ir()
                if not (bl > threshold and br > threshold):
                    break
            self.zumi.reset_gyro()
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, 0)
                if br > threshold and bl > threshold:
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 15)
            self.zumi.turn_right(25)

        else:
            # No line on left → try right
            print("no line left")
            turned_right_angle = self.turn_to_check("right", 80)
            _, br, _, bl, _, _ = self.read_ir()

            while bl > threshold and br > threshold:
                self.zumi.turn_left(5)
                _, br, _, bl, _, _ = self.read_ir()
            self.zumi.reset_gyro()
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, 0)
                if br > threshold and bl > threshold:
                    print("line found")
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 15)
            self.zumi.turn_left(25)

        self.zumi.reset_gyro()

    def find_line(self, angle_to_turn):
        """
        After a 360° dance, realign with the line:
        1. Move until both bottom sensors < threshold (off-line).
        2. Move until both bottom sensors > threshold (back on-line).
        3. If bounce < 0.3 s → correct path and go to finish.
        4. Else if bounce < 1 s → go to finish.
        """
        self.zumi.reset_gyro()
        speed = 5
        threshold = 100

        # 1) Move until off the line
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br < threshold and bl < threshold:
                self.zumi.stop()
                break

        # 2) Move until back on the line
        while True:
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br > threshold and bl > threshold:
                self.zumi.stop()
                break

        self.logger.log_event("again_on_line")
        start_time = time.time()

        # 3) Up to 1 s to see if we immediately go off→on again
        while True:
            if time.time() - start_time > 1:
                self.zumi.stop()
                print("time is out")
                return
            _, br, _, bl, _, _ = self.read_ir()
            self.zumi.go_straight(speed, angle_to_turn)
            if br < threshold and bl < threshold:
                self.zumi.stop()
                break

        self.logger.log_event("out_of_line")
        dt = (self.logger.log["out_of_line"][-1] - self.logger.log["again_on_line"][-1]).total_seconds()

        if dt < 0.3:
            # Bounce < 0.3 s → correct then go to finish
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, angle_to_turn)
                if br > threshold and bl > threshold:
                    self.zumi.stop()
                    break
            while True:
                _, br, _, bl, _, _ = self.read_ir()
                self.zumi.go_straight(speed, angle_to_turn)
                if br < threshold and bl < threshold:
                    self.zumi.stop()
                    break
            self.move_after_turn(speed, 0, 3)
            self.find_finish_line()

        elif dt < 1:
            # Bounce < 1 s → одразу до finish
            print("find_line_2")
            self.move_after_turn(speed, 0, 3)
            self.find_finish_line()

    def dance_360(self, direction, times):
        """
        Perform a 'dance' by spinning 360° in the specified direction, `times` times.
        Returns the final remainder angle.
        """
        result_angle = 0
        for _ in range(times):
            self.zumi.reset_gyro()
            if direction == "left":
                self.zumi.signal_left_on()
                angle_to_turn = 360
                while angle_to_turn > 10:
                    self.zumi.turn_left(angle_to_turn)
                    angle_to_turn = 360 - self.zumi.read_z_angle()
                self.zumi.signal_left_off()
                self.zumi.stop()
                result_angle = angle_to_turn
            else:
                self.zumi.signal_right_on()
                angle_to_turn = 360
                while angle_to_turn > 10:
                    self.zumi.turn_right(angle_to_turn)
                    angle_to_turn = 360 + self.zumi.read_z_angle()
                self.zumi.signal_right_off()
                self.zumi.stop()
                result_angle = angle_to_turn
        return result_angle

    def finish_with_180(self):
        """
        Perform a final 180° turn and log "finish" event.
        """
        self.zumi.stop()
        self.logger.log_event("finish")
        print("Reached end. Performing 180° turn.")
        self.screen.draw_text_center("Finisher box\nTurning 180°")
        self.zumi.turn_left(180)
        self.screen.draw_text_center("Done!")

    def go_straight(self, speed, desired_angle):
        """
        Wrapper around the low-level zumi.go_straight() call.
        """
        self.zumi.go_straight(speed, desired_angle)

    def run(self, qr_handler):
        """
        Main loop implementing exactly the requested logic:
        1) Move forward and constantly check if an object is ahead.
        2) If an object is detected, wait until it is removed (i.e., front sensors > threshold).
        3) Then turn on camera and up to 5 frames check for QR-code or face:
           - If found QR → execute corresponding action via qr_handler.
           - If found face → save image, increment face count.
        4) After handling object (even if action was only emotion), resume moving straight
           until reaching the “end” of the line (both bottom sensors > threshold).
        5) When “end” of line is reached:
           a) Turn left once, then check:
              • If new line detected (bottom sensors < threshold) – drive straight.
              • Else → turn right, check again:
                 ◦ If line now detected – drive straight.
                 ◦ Else → turn left again and move backward until line is found.
        """
        threshold = 100
        speed_forward = 4
        number_of_faces = 0

        # 1) Move forward and check for object
        while True:
            front_right, _, _, _, _, front_left = self.read_ir()
            if front_right < threshold and front_left < threshold:
                # Object detected
                self.logger.log_event("object_detected")
                self.zumi.stop()

                # 2) Wait until object is removed
                while True:
                    front_right, _, _, _, _, front_left = self.read_ir()
                    if front_right > threshold or front_left > threshold:
                        break
                    time.sleep(0.1)

                # 3) Now check camera up to 5 frames for QR or face
                self.camera.start_camera()
                for _ in range(5):
                    frame = self.camera.capture()
                    self.camera.show_image(frame)

                    # Try QR first
                    qr_message = self.vision.find_QR_code(frame)
                    if qr_message is not None:
                        self.camera.close()
                        decoded = self.vision.get_QR_message(qr_message)
                        self.screen.draw_text_center("QR Code Detected!")
                        self.logger.log_event(f"qr_code_command:{decoded}")
                        # Dispatch QR action
                        handler = qr_handler.QR_HANDLERS.get(decoded)
                        if handler:
                            handler({
                                "speed": speed_forward,
                                "number_of_objects": None,  # not needed here
                                "threshold": threshold
                            })
                            self.logger.log_event(f"qr_code_command:{decoded}_done")
                        else:
                            if "360" in str(decoded):
                                qr_handler._handle_360_message(decoded)
                        break

                    # If no QR, try face detection
                    gray = self.vision.convert_to_gray(frame)
                    if self.vision.find_face(gray) is not None:
                        self.camera.close()
                        self.screen.draw_text_center("Face Detected!")
                        self.logger.log_event("face_detected")
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"{timestamp}.png"
                        cv2.imwrite(filename, gray)
                        number_of_faces += 1
                        break

                    time.sleep(1)
                self.camera.close()

                # 4) After handling, resume moving forward until “end” of line
                while True:
                    _, bottom_right, _, bottom_left, _, _ = self.read_ir()
                    if bottom_right > threshold and bottom_left > threshold:
                        self.zumi.stop()
                        self.logger.log_event("end_of_line_reached")
                        break
                    self.go_straight(speed_forward, 0)

                # 5) “End” of line logic:
                #   a) Turn left and check if line present
                self.turn_90("left")
                _, br, _, bl, _, _ = self.read_ir()
                if bl < threshold and br < threshold:
                    # Found line on left → drive straight
                    self.logger.log_event("line_found_after_left_turn")
                    continue  # back to moving straight until next object

                #   b) If no line on left → turn right and check
                self.turn_90("right")  # This effectively makes a net turn right relative to original heading
                _, br, _, bl, _, _ = self.read_ir()
                if bl < threshold and br < threshold:
                    self.logger.log_event("line_found_after_right_turn")
                    continue  # back to moving straight

                #   c) If still no line → turn left back (relative heading), then go backward until find line
                self.turn_90("left")  # Now heading is 180° from original direction
                while True:
                    _, br, _, bl, _, _ = self.read_ir()
                    if bl < threshold and br < threshold:
                        self.logger.log_event("line_found_while_moving_backward")
                        break
                    # Move backwards: (negative speed or inverse command depending on API)
                    # Якщо API Зумі не дозволяє подавати негативний speed, можна симулювати
                    # дублюючи рух через go_straight з кутом 180°, або додати метод move_backward.
                    self.zumi.go_straight(-speed_forward, 0)
                    time.sleep(0.05)
                # Після знаходження лінії автоматично почнеться новий цикл → рух уперед
            else:
                # Нема об'єкта → просто рухатись прямо
                self.go_straight(speed_forward, 0)

        # Поки в цьому прикладі ми ніколи не виходимо з зовнішнього while True,
        # але якщо необхідно, тут можна викликати finish_with_180 і save_to_csv.

In [ ]:
# 📷 4. QRCodeFaceHandler CLASS

class QRCodeFaceHandler:
    """
    Handles detection of QR codes or faces using the camera,
    and triggers appropriate ZumiDriver methods for each recognized command.
    """

    def __init__(self, driver, logger):
        """
        Args:
            driver (ZumiDriver): instance of ZumiDriver
            logger (EventLogger): instance of EventLogger
        """
        self.driver = driver
        self.logger = logger

        # Dispatcher dict for QR messages → handler methods
        self.QR_HANDLERS = {
            "Left Circle": self._cmd_left_circle,
            "Right Circle": self._cmd_right_circle,
            "Turn Left": self._cmd_turn_left,
            "Turn Right": self._cmd_turn_right,
            "Stop": self._cmd_stop,
            "Zumi is happy today!": self._cmd_happy,
            "Zumi is angry today!": self._cmd_angry,
            "Zumi is celebrating today!": self._cmd_celebrate,
        }

    def _cmd_left_circle(self, args: dict):
        """
        Handler for "Left Circle" QR command.
        """
        number_of_objects = args.get("number_of_objects", 1)
        threshold = args.get("threshold", 100)
        speed = args.get("speed", 4)
        self.driver.circle("left", number_of_objects, threshold, speed)

    def _cmd_right_circle(self, args: dict):
        """
        Handler for "Right Circle" QR command.
        """
        number_of_objects = args.get("number_of_objects", 1)
        threshold = args.get("threshold", 100)
        speed = args.get("speed", 4)
        self.driver.circle("right", number_of_objects, threshold, speed)

    def _cmd_turn_left(self, args: dict):
        """
        Handler for "Turn Left" QR command.
        """
        self.driver.zumi.signal_left_on()
        self.driver.zumi.turn_left(90)
        self.driver.zumi.signal_left_off()

    def _cmd_turn_right(self, args: dict):
        """
        Handler for "Turn Right" QR command.
        """
        self.driver.zumi.signal_right_on()
        self.driver.zumi.turn_right(90)
        self.driver.zumi.signal_right_off()

    def _cmd_stop(self, args: dict):
        """
        Handler for "Stop" QR command.
        """
        self.driver.zumi.stop()

    def _cmd_happy(self, args: dict):
        """
        Handler for "Zumi is happy today!" QR command.
        """
        self.driver.personality.happy()

    def _cmd_angry(self, args: dict):
        """
        Handler for "Zumi is angry today!" QR command.
        """
        self.driver.personality.angry()

    def _cmd_celebrate(self, args: dict):
        """
        Handler for "Zumi is celebrating today!" QR command.
        """
        self.driver.personality.celebrate()

    def _handle_360_message(self, message: str):
        """
        Parse and handle 360° spin messages, e.g., "3 times spin 360 left,happy".
        Format: "<n> times spin 360 <direction>,<emotion>"
        """
        parts = message.replace(",", " ").split()
        # Example: ["3", "times", "spin", "360", "left", "happy"]
        try:
            times = int(parts[0])
            direction = parts[4]
            emotion = parts[5]
        except (IndexError, ValueError):
            print("Invalid 360 message format")
            return

        angle_to_turn = self.driver.dance_360(direction, times) * 2

        if emotion == "happy":
            self.driver.personality.happy()
        elif emotion == "angry":
            self.driver.personality.angry()
        elif emotion == "celebrating":
            self.driver.personality.celebrate()

        self.driver.zumi.stop()
        print(f"Angle to turn: {angle_to_turn}")
        self.driver.find_line(angle_to_turn)

    def process_object(self, speed, number_of_objects, number_of_faces, threshold):
        """
        Called when an object is detected.
        Attempts up to 5 frames to find a QR code or a face.
        Returns the updated number_of_faces (incremented if face found).
        """
        self.driver.camera.start_camera()

        for _ in range(5):
            frame = self.driver.camera.capture()
            self.driver.camera.show_image(frame)

            qr_message = self._check_qr_code(frame)
            if qr_message is not None:
                self.driver.camera.close()
                self.driver.screen.draw_text_center("QR Code Detected!")
                self.logger.log_event(f"qr_code_command:{qr_message}")
                print("QR code detected!")

                handler = self.QR_HANDLERS.get(qr_message)
                if handler:
                    handler(
                        {
                            "speed": speed,
                            "number_of_objects": number_of_objects,
                            "threshold": threshold,
                        }
                    )
                    self.logger.log_event(f"qr_code_command:{qr_message}_done")
                else:
                    if "360" in str(qr_message):
                        self._handle_360_message(qr_message)
                    else:
                        print("Invalid QR command")
                break  # stop after handling QR

            # If no QR, convert to gray and check face
            gray = self.driver.vision.convert_to_gray(frame)
            if self._check_face(gray):
                self.driver.camera.close()
                self.driver.screen.draw_text_center("Face Detected!")
                self.logger.log_event("face_detected")
                print("Face detected!")
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                filename = f"{timestamp}.png"
                cv2.imwrite(filename, gray)
                number_of_faces += 1
                break

            time.sleep(1)

        self.driver.camera.close()
        return number_of_faces

    def _check_qr_code(self, frame):
        """
        Try to detect and decode a QR code in the given frame.
        Returns:
            Decoded message (str) or None if not found.
        """
        qr_code = self.driver.vision.find_QR_code(frame)
        if qr_code is not None:
            return self.driver.vision.get_QR_message(qr_code)
        return None

    def _check_face(self, gray_frame):
        """
        Detect a face in a grayscale image.
        Returns True if found, else False.
        """
        return self.driver.vision.find_face(gray_frame) is not None

In [ ]:
# 🏃 5. MAIN LOOP (RUN)

# Instantiate mocks or real APIs
# For demonstration, використаємо «Mock» обʼєкти,
# але в реальному середовищі замініть їх на справжні з Zumi SDK.
zumi =    Zumi()
camera    = Camera()
screen    = Screen()
vision    = Vision()
personality = Personality(zumi, screen)
logger = EventLogger()

# Ініціалізуємо драйвер і обробника QR/face
driver = ZumiDriver(
    zumi=zumi,
    camera=camera,
    vision=vision,
    screen=screen,
    personality=personality,
    logger=logger,
)
qr_handler = QRCodeFaceHandler(driver=driver, logger=logger)

# Запускаємо основний цикл (цю клітинку можна зупинити вручну, наприклад, клавішею «Stop» у Jupyter)
driver.run(qr_handler)

In [ ]:
# 💾 7. SAVE LOGS

output_folder = "logs"
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

logger.save_to_csv(output_folder=output_folder)
print("Notebook execution completed.")